<a href="https://colab.research.google.com/github/MattLMerritt/protocol-protocol/blob/stateful_framework/visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install networkx
# ! pip install matplotlib
! pip install pyvis

How to to use:

upload steps_format file. This file contains information on the amount of steps to visualize, the amount of devices, and the amount of wires. In this current version devices and wires are only refered to as their ID which starts from 0 and increments up. The structure of how the wires are connected to the devices needs to be hardcoded at the moment.

```
STEP EVENT FORMAT
{
    "steps": 10,
    "devices": 2,
    "wires": 1,
    "step-0": {
        "d-1": "IDLE",
        "d-2": "IDLE",
        "w-1": "EMPTY"
    },
    more steps...
}
```

In [10]:
# part 1: parse step input file
import json
step_file_dict = json.load(open("test.json"))

simulation_steps = step_file_dict["steps"]
amount_of_devices = step_file_dict["devices"]
amount_of_wires = step_file_dict["wires"]

for step_index in range(simulation_steps):
  print("\nstep: " + str(step_index))
  step = step_file_dict["step-" + str(step_index)]
  
  # loop through each device's state in this step
  for device_index in range(amount_of_devices):
    device_state = step["d-" + str(device_index)]
    print("device-" + str(device_index) + " state: " + device_state)

  # loop through each wire's state in this step
  for wire_index in range(amount_of_wires):
    wire_state = step["w-" + str(wire_index)]
    print("wire-" + str(wire_index) + " state: " + wire_state)


step: 0
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY

step: 1
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY

step: 2
device-0 state: SENDING
device-1 state: IDLE
wire-0 state: IN_USE

step: 3
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: IN_USE

step: 4
device-0 state: IDLE
device-1 state: RECEIVING
wire-0 state: IN_USE

step: 5
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY

step: 6
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY

step: 7
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY

step: 8
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY

step: 9
device-0 state: IDLE
device-1 state: IDLE
wire-0 state: EMPTY
